# Test orbit file (LED) reading and Hermite interpolation

In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.lines import Line2D
%matplotlib inline

In [3]:
from pygmtsar import PRM
from scipy.interpolate import CubicHermiteSpline

### Load Parameters and Orbits

In [4]:
prm_ref = PRM.from_file('S1_20201222_ALL_F2.PRM')
prm_rep = PRM.from_file('S1_20210103_ALL_F2.PRM')

In [5]:
orb_ref = prm_ref.read_LED()
orb_rep = prm_rep.read_LED()

### Reference

In [6]:
t_ref = prm_ref.get_seconds()
t_ref = [t_ref[0], np.sum(t_ref)/2, t_ref[-1]]
t_ref[0] - 30811049.791369, t_ref[1] - 30811051.287814, t_ref[2] - 30811052.784259
# % ./SAT_baseline.sh
# using command line
# SC_identity = 10 
# ......master LED file S1_20201222_ALL_F2.LED 
# .........aligned LED file S1_20210103_ALL_F2.LED 
# 	reference orbit times: 30811049.791369, 30811052.784259, 30811051.287814
# 	repeat orbit times: 225449.207167, 225452.200057, 225450.703612

(-1.5273690223693848e-07, -1.6391277313232422e-07, -1.7881393432617188e-07)

In [7]:
t_ref[-1] - t_ref[0]

2.9928899742662907

In [8]:
print(orb_ref.attrs)
orb_ref

{'nd': '281', 'iy': '2020', 'id': '356', 'isec': '51252.000000', 'idsec': '10.000'}


,iy,id,isec,px,py,pz,vx,vy,vz,clock
0,2020,356,51252.0,2.126633e+06,4.552170e+06,-4.989856e+06,3884.019377,3895.866320,5213.514095,30809652.0
1,2020,356,51262.0,2.165382e+06,4.590845e+06,-4.937442e+06,3865.729372,3839.235712,5269.275977,30809662.0
2,2020,356,51272.0,2.203946e+06,4.628953e+06,-4.884473e+06,3846.923435,3782.199871,5324.450320,30809672.0
3,2020,356,51282.0,2.242319e+06,4.666488e+06,-4.830955e+06,3827.603002,3724.765828,5379.030847,30809682.0
4,2020,356,51292.0,2.280496e+06,4.703447e+06,-4.776894e+06,3807.769575,3666.940659,5433.011344,30809692.0
...,...,...,...,...,...,...,...,...,...,...
276,2020,356,54012.0,-2.069327e+06,-3.293189e+06,5.901384e+06,-5109.703461,-3947.281735,-3985.399903,30812412.0
277,2020,356,54022.0,-2.120336e+06,-3.332440e+06,5.861198e+06,-5091.966582,-3902.738810,-4051.757205,30812422.0
278,2020,356,54032.0,-2.171164e+06,-3.371243e+06,5.820351e+06,-5073.593365,-3857.783983,-4117.659359,30812432.0
279,2020,356,54042.0,-2.221806e+06,-3.409594e+06,5.778847e+06,-5054.585233,-3812.423186,-4183.098903,30812442.0


In [9]:
print(orb_rep.attrs)
orb_rep

{'nd': '281', 'iy': '2021', 'id': '2', 'isec': '51252.000000', 'idsec': '10.000'}


,iy,id,isec,px,py,pz,vx,vy,vz,clock
0,2021,2,51252.0,2.128971e+06,4.554425e+06,-4.986805e+06,3882.942455,3892.555870,5216.801202,224052.0
1,2021,2,51262.0,2.167709e+06,4.593067e+06,-4.934358e+06,3864.621540,3835.901754,5272.528762,224062.0
2,2021,2,51272.0,2.206262e+06,4.631141e+06,-4.881357e+06,3845.784778,3778.842819,5327.668412,224072.0
3,2021,2,51282.0,2.244623e+06,4.668643e+06,-4.827807e+06,3826.433611,3721.386098,5382.213880,224082.0
4,2021,2,51292.0,2.282789e+06,4.705568e+06,-4.773714e+06,3806.569545,3663.538671,5436.158955,224092.0
...,...,...,...,...,...,...,...,...,...,...
276,2021,2,54012.0,-2.072360e+06,-3.295447e+06,5.899060e+06,-5108.682789,-3944.684131,-3989.299038,226812.0
277,2021,2,54022.0,-2.123359e+06,-3.334672e+06,5.858835e+06,-5090.908112,-3900.117442,-4055.630120,226822.0
278,2021,2,54032.0,-2.174176e+06,-3.373448e+06,5.817949e+06,-5072.497178,-3855.139196,-4121.505617,226832.0
279,2021,2,54042.0,-2.224807e+06,-3.411773e+06,5.776406e+06,-5053.451414,-3809.755328,-4186.918069,226842.0


In [10]:
#reference orbit first point:  2126632.873229, 4552169.567944, -4989856.255835
#orb[['px', 'py', 'pz']].head(1).values
spline_ref = CubicHermiteSpline(orb_ref['clock'], orb_ref[['px', 'py', 'pz']].values, orb_ref[['vx', 'vy', 'vz']].values)
spline_rep = CubicHermiteSpline(orb_rep['clock'], orb_rep[['px', 'py', 'pz']].values, orb_rep[['vx', 'vy', 'vz']].values)

spline_ref(orb_ref['clock'].values[0]) - np.array([ 2126632.873229,  4552169.567944, -4989856.255835])

array([0., 0., 0.])

In [11]:
# reference orbit start point:  3934597.486830, 3828232.879869, 4455902.914510
spline_ref(t_ref[0]) - np.array([3934597.486830, 3828232.879869, 4455902.914510])

array([-8.16844404e-05, -5.17568551e-05, -7.23702833e-05])

In [12]:
# reference orbit center point:    3931447.623914, 3821459.815849, 4464465.289022
spline_ref(t_ref[1]) - np.array([3931447.623914, 3821459.815849, 4464465.289022])

array([-1.23162754e-05, -7.54557550e-06, -1.10631809e-05])

In [13]:
#reference orbit end point: 3928286.413630, 3814677.847949, 4473016.374198
spline_ref(t_ref[2]) - np.array([3928286.413630, 3814677.847949, 4473016.374198])

array([-1.37868337e-05, -8.89413059e-06, -1.25141814e-05])

### Repeat

In [14]:
#repeat orbit times: 225449.207167, 225452.200057, 225450.703612
t_rep = prm_rep.get_seconds()
t_rep = [t_rep[0], np.sum(t_rep)/2, t_rep[-1]]
t_rep[0] - 225449.207167, t_rep[1] - 225450.703612, t_rep[2] - 225452.200057

(5.352194420993328e-08, 4.0832674130797386e-08, 2.814340405166149e-08)

## TODO

In [ ]:
# void calc_height_velocity(
#     struct SAT_ORB *orb,    // Pointer to a structure containing satellite orbit information.
#     struct PRM *prm,        // Pointer to a structure containing parameters for processing.
#     double t1,              // Start time for the calculations.
#     double t2,              // End time for the calculations.
#     double *height,         // Pointer to a double where the calculated height will be stored.
#     double *re2,            // Pointer to a double where the calculated Earth's radius will be stored.
#     double *vg,             // Pointer to a double where the calculated ground velocity will be stored.
#     double *vtot,           // Pointer to a double where the calculated total velocity will be stored.
#     double *rdot            // Pointer to a double where the calculated range rate will be stored.
# );

## SAT_baseline

In [15]:
# Interpolating positions for reference orbit
pos_ref = spline_ref(t_ref)
# Interpolating positions for repeat orbit
pos_rep = spline_rep(t_rep)

# Compute the initial distances
def compute_initial_distances(prm, spline, t_ref, ref_pos):
    ns2 = int(prm.get('nrows') * 0.5)
    dt = 0.5 / prm.get('PRF')
    times = t_ref + dt * range(-ns2, ns2)
    #print ('compute_initial_distances times:', len(times), times)
    positions = spline(times)
    dists = np.linalg.norm(positions - ref_pos, axis=1)
    idx = np.argmin(dists)
    return times[idx], positions[idx], dists[idx]

# Polynomial refinement
def poly_interp(spline, t_ref, ref_pos):
    ntt = 100
    ddt = 0.01 / ntt
    times = np.array([(k - ntt / 2 + 0.5) * ddt for k in range(ntt)])
    #print ('def poly_interp(spline, t_ref, ref_pos): times:', len(times), times[:3], '...', times[-3:])
    positions = spline(t_ref + times)
    distances = np.linalg.norm(positions - ref_pos, axis=1)**2
    p = np.polyfit(times, distances, 2)
    t = t_ref - p[1] / (2.0 * p[0])
    b = np.sqrt(p[2] - p[1] ** 2 / (4.0 * p[0]))
    return t, spline(t), b

ht = []
B_offset = []
baseline = []
alpha = []
for idx in range(3):
    height, re_c = prm_rep.get_height(*spline_rep(t_rep[idx]))
    ht.append(height + re_c - prm_rep.get('earth_radius'))
    # find nearest points in the repeat orbit for the start, center, and end of the reference orbit
    ts, pos_rep = compute_initial_distances(prm_ref, spline_rep, t_rep[idx], pos_ref[idx])[:2]
    # refine baseline computation
    ts, pos_rep = poly_interp(spline_rep, ts, pos_ref[idx])[:2]
    B_offset.append( (ts - t_rep[idx]) * prm_rep.get('SC_vel') )
    # baseline components
    baseline.append( pos_rep - pos_ref[idx] )
    bv, bh = prm_ref.get_components(baseline[idx],  pos_ref[idx], pos_rep)
    alpha.append( np.degrees(np.arctan2(bv, bh)) )

bpara, bperp = prm_ref.get_baseline_projections(prm_rep, baseline[0], alpha[0])

# TODO: tie_point, look vectors, shifts
# find expected offset in pixels (rshift and yshift)

print(f"B_parallel = {bpara}, B_perpendicular = {bperp}")
print(f"Baseline at start:  total {np.linalg.norm(baseline[0])}, alpha = {alpha[0]}")
print(f"Baseline at center: total {np.linalg.norm(baseline[1])}, alpha = {alpha[1]}")
print(f"Baseline at end:    total {np.linalg.norm(baseline[2])}, alpha = {alpha[2]}")
print(f"B_offset_start = {B_offset[0]}, B_offset_center = {B_offset[1]}, B_offset_end = {B_offset[2]}")
# B_parallel          = 0.153127184931 
# B_perpendicular     = 9.126229701452 
# baseline_start      = 9.127514256271 
# baseline_center     = 9.206174906760 
# baseline_end        = 9.285362702893 
# alpha_start         = 38.060991577255 
# alpha_center        = 37.618029103013 
# alpha_end           = 37.182533686550 
# B_offset_start      = -4.908882721268 
# B_offset_center     = -4.887323045248 
# B_offset_end        = -4.865815416072 

#repeat orbit (ts - t21), vel: -0.000686, 7153.263621

ht[2] - 700857.378603, ht[1] - 700872.176433, ht[0] - 700886.984341,
bpara - 0.153127184931, bperp - 9.126229701452, \
np.linalg.norm(baseline[0])  - 9.127514256271, \
np.linalg.norm(baseline[1]) - 9.206174906760, \
np.linalg.norm(baseline[2])    - 9.285362702893, \
B_offset[0] - -4.908882721268, B_offset[1] - -4.887323045248, B_offset[2] - -4.865815416072, \
alpha[0] - 38.060991577255, alpha[1] - 37.618029103013, alpha[2] - 37.182533686550

B_parallel = 0.1531673895604653, B_perpendicular = 9.126210710508152
Baseline at start:  total 9.127495942580225, alpha = 38.06073725529894
Baseline at center: total 9.206160182341616, alpha = 37.6178342369064
Baseline at end:    total 9.285371571510327, alpha = 37.18263652774751
B_offset_start = -4.9088783493327055, B_offset_center = -4.887321379749086, B_offset_end = -4.865816040633634


(4.0204629465301345e-05,
 -1.899094384860689e-05,
 -1.831369077542888e-05,
 -1.4724418383238458e-05,
 8.868617326740491e-06,
 4.371935294678053e-06,
 1.6654989138942256e-06,
 -6.245616335220916e-07,
 -0.00025432195606356345,
 -0.0001948661066037971,
 0.00010284119750991749)

In [16]:
import scipy
import numpy as np
#from scipy.constants import speed_of_light

from pyproj import Proj, Transformer

# Initialize WGS84 coordinate system
wgs84 = Proj(proj='latlong', datum='WGS84')
wgs84_geocentric = Proj(proj='geocent', datum='WGS84')

# Initialize the Transformer
transformer_to_geocentric = Transformer.from_proj(wgs84, wgs84_geocentric)
transformer_to_geodetic = Transformer.from_proj(wgs84_geocentric, wgs84)

# Function to convert from ellipsoidal to Cartesian coordinates (plh to xyz)
def plh2xyz(lat, lon, h):
    x, y, z = transformer_to_geocentric.transform(lon, lat, h)
    return np.array([x, y, z])

# Function to convert from Cartesian to ellipsoidal coordinates (xyz to plh)
def xyz2plh(x, y, z):
    lon, lat, h = transformer_to_geodetic.transform(x, y, z)
    return lat, lon, h

In [17]:
# Function to compute unit vectors
def compute_unit_vectors(position):
    ru = np.linalg.norm(position)
    xu = position[0] / ru
    yu = position[1] / ru
    zu = position[2] / ru
    return ru, xu, yu, zu

# Calculate tie point and look vectors
def calculate_tie_point_and_look_vectors(prm_ref, prm_rep, t_sta, t_vel, ref_pos_start, spline_ref):
    # Get the necessary parameters from PRM
    #earth_radius = prm_ref.get('earth_radius')
    #near_range = prm_ref.get('near_range')
    #ht = prm_ref.get('SC_height')
    dr = 0.5 * scipy.constants.speed_of_light / prm_ref.get('rng_samp_rate')
    far_range = prm_ref.get('near_range') + dr * prm_ref.get('num_rng_bins')
    rho = (prm_ref.get('near_range') + far_range) / 2
    
    # Radar coordinates
    b = prm_ref.get('SC_height') + prm_ref.get('earth_radius')
    theta = np.arccos((b**2 + rho**2 - prm_ref.get('earth_radius')**2) / 2 / b / rho)
    #print ('theta', theta)

    radar_look = np.zeros(3)
    radar_look[1] = np.cos(theta)
    radar_look[2] = -np.sin(theta)
    if prm_ref.get('lookdir') == 'L':
        radar_look[2] = np.sin(theta)
    
    # Get the satellite position and velocity at t_sta and t_vel
    x_sta, y_sta, z_sta = spline_ref(t_sta)
    x_vel, y_vel, z_vel = spline_ref(t_vel)
    
    # Compute unit vectors
    ru_sta, xu_sta, yu_sta, zu_sta = compute_unit_vectors(np.array([x_sta, y_sta, z_sta]))
    ru_vel, xu_vel, yu_vel, zu_vel = compute_unit_vectors(np.array([x_vel - x_sta, y_vel - y_sta, z_vel - z_sta]))
    
    o1y = np.array([-xu_sta, -yu_sta, -zu_sta])
    o1x = np.array([xu_vel, yu_vel, zu_vel])
    
    o1z = np.cross(o1x, o1y)
    
    # Look vector in global cartesian coordinates
    glob_look = radar_look[0] * o1x + radar_look[1] * o1y + radar_look[2] * o1z
    
    # Compute the target in global cartesian coordinates
    target = np.array([x_sta, y_sta, z_sta]) + glob_look * rho
    
    # Convert to ellipsoidal coordinates
    lat, lon, h = xyz2plh(target[0], target[1], target[2])
    target_llt = np.array([lat, lon, h])
    
    return target, target_llt

# Determine tie point, look vectors, shifts
t_sta = t_ref[0] + 2.0
t_vel = t_ref[0] + 2.1
#print ('t_sta', t_sta, 't_vel', t_vel)

target, target_llt = calculate_tie_point_and_look_vectors(prm_ref, prm_rep, t_sta, t_vel, pos_ref[0], spline_ref)
lon_tie_point = (target_llt[1] > 180.0) * (target_llt[1] - 360.0) + (target_llt[1] <= 180.0) * target_llt[1]
lat_tie_point = target_llt[0]

print(f"lon_tie_point = {lon_tie_point}")
print(f"lat_tie_point = {lat_tie_point}")

print (target_llt[1] - 49.873465, target_llt[0] - 40.286379)

lon_tie_point = 49.8734654788201
lat_tie_point = 40.28637916813154
4.788200982375201e-07 1.6813154246619888e-07


In [18]:
# Function to convert latitude, longitude, and height to range, azimuth, and height
def llt2rat_sub(prm, spline, target_llt):
    vel = prm.get('SC_vel')

    # Convert target lat/lon/height to xyz (xp variable)
    target_xyz = plh2xyz(target_llt[0], target_llt[1], target_llt[2])

    # Compute start and stop time
    t1 = 86400.0 * prm.get('clock_start') + (prm.get('nrows') - prm.get('num_valid_az')) / (2.0 * prm.get('PRF'))
    t2 = t1 + prm.get('num_patches') * prm.get('num_valid_az') / prm.get('PRF')
    # Sampling the orbit every 2nd point
    ts = 2.0 / prm.get('PRF')
    nrec = int((t2 - t1) / ts)

    # orbit positions
    # /* number of buffer points to add before and after the acquisition */
    npad = 8000
    orb_time = t1 - ts * npad + ts * range(nrec + npad * 2);
    #print ('orb_time', len(orb_time))    
    orb_pos = spline(orb_time)

    dists = np.linalg.norm(orb_pos - target_xyz, axis=1)
    idx = np.argmin(dists)
    rng = dists[idx]
    tm = orb_time[idx]

    # Compute range and azimuth in pixel space and correct for azimuth bias
    dr = 0.5 * scipy.constants.speed_of_light / prm.get('rng_samp_rate')
    rng_pix = (rng - prm.get('near_range')) / dr - (prm.get('rshift') + prm.get('sub_int_r')) + prm.get('chirp_ext')
    az_pix = prm.get('PRF') * (tm - t1) - (prm.get('ashift') + prm.get('sub_int_a'))

    # Azimuth and range correction if Doppler is not zero
    if prm.get('fd1') != 0.0:
        rdd = (vel * vel) / rng
        daa = -0.5 * (prm.get('radar_wavelength') * fd1) / rdd
        drr = 0.5 * rdd * daa * daa / dr
        daa = prf * daa
        rng_pix += drr
        az_pix += daa

    target_rat = np.array([rng_pix, az_pix, np.linalg.norm(target_xyz) - prm.get('earth_radius')])
    return target_rat

target_rat_ref = llt2rat_sub(prm_ref, spline_ref, target_llt)
target_rat_rep = llt2rat_sub(prm_rep, spline_rep, target_llt)
#print ('target_rat_ref', target_rat_ref)
#print ('target_rat_rep', target_rat_rep)

# # Compute expected offset in pixels (rshift and yshift)
ashift = int(target_rat_rep[1] - target_rat_ref[1])
rshift = int(target_rat_rep[0] - target_rat_ref[0])

print(f"ashift = {ashift}, rshift = {rshift}")

ashift = 0, rshift = 0


In [19]:
#llt2rat_sub target_rat:  12794.831537, 1038.380102, -5.071846
#llt2rat_sub target_rat:  12794.614100, 1041.773079, -5.071846
target_rat_ref[0] - 12794.831537, target_rat_ref[1] - 1038.380102, target_rat_ref[2] - -5.071846, \
target_rat_rep[0] - 12794.614100, target_rat_rep[1] - 1041.773079, target_rat_rep[2] - -5.071846

(-0.00016314880667778198,
 2.0000007996870863,
 -2.0972573162580943e-06,
 -0.0001307832390011754,
 -2.0000000056288627,
 -2.0972573162580943e-06)

In [20]:
PRM().set(
    lon_tie_point=lon_tie_point,
    lat_tie_point=lat_tie_point,
    SC_vel='TODO',
    SC_height=ht[1],
    SC_height_start=ht[0],
    SC_height_end=ht[2],
    earth_radius=prm_rep.get('earth_radius'),
    rshift=rshift,   
    sub_int_r=0.0,
    ashift=ashift,
    sub_int_a=0.0,
    B_parallel=bpara,
    B_perpendicular=bperp,
    baseline_start=np.linalg.norm(baseline[0]),
    baseline_center=np.linalg.norm(baseline[1]),
    baseline_end=np.linalg.norm(baseline[2]),
    alpha_start=alpha[0],
    alpha_center=alpha[1],
    alpha_end=alpha[2],
    B_offset_start=B_offset[0],
    B_offset_center=B_offset[1],
    B_offset_end=B_offset[2]
)
# lon_tie_point =  49.873465
# lat_tie_point =  40.286379
# SC_vel              = 7153.263621000000 
# SC_height           = 700872.176433234476 
# SC_height_start     = 700886.984341199510 
# SC_height_end       = 700857.378603472374 
# earth_radius        = 6369585.038519999944 
# rshift              = 0 
# sub_int_r           = 0.0 
# ashift              = 3
# sub_int_a           = 0.0 
# B_parallel          = 0.153127184931 
# B_perpendicular     = 9.126229701452 
# baseline_start      = 9.127514256271 
# baseline_center     = 9.206174906760 
# baseline_end        = 9.285362702893 
# alpha_start         = 38.060991577255 
# alpha_center        = 37.618029103013 
# alpha_end           = 37.182533686550 
# B_offset_start      = -4.908882721268 
# B_offset_center     = -4.887323045248 
# B_offset_end        = -4.865815416072 

Object PRM 22 items
                         value
name                          
lon_tie_point        49.873465
lat_tie_point        40.286379
SC_vel                    TODO
SC_height        700872.176382
SC_height_start  700886.984179
SC_height_end    700857.378602
earth_radius     6369585.03852
rshift                       0
sub_int_r                  0.0
ashift                       0
sub_int_a                  0.0
B_parallel            0.153167
B_perpendicular       9.126211
baseline_start        9.127496
baseline_center        9.20616
baseline_end          9.285372
alpha_start          38.060737
alpha_center         37.617834
alpha_end            37.182637
B_offset_start       -4.908878
B_offset_center      -4.887321
B_offset_end         -4.865816

In [22]:
%timeit transformer_to_geocentric.transform(0, 0, 0)

519 ns ± 4.76 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [91]:

def get_velocity(self, spline, t1, t2):
    # ellipsoid parameters
    ra = self.get('equatorial_radius')
    rc = self.get('polar_radius')

    dt = 200.0 / self.get('PRF')
    # nt = (prm->nrows - prm->num_valid_az)/100.
	# changed to fixed value for TSX, ENVI
    nt = 100

    t0 = (t1 + t2) / 2.0
    t1 = t0 - 2.0
    t2 = t0 + 2.0

    xs, ys, zs = spline(t0)
    x1, y1, z1 = spline(t1)
    x2, y2, z2 = spline(t2)
    #print ('xs, ys, zs', xs, ys, zs)
    
    vx = (x2 - x1) / 4.0
    vy = (y2 - y1) / 4.0
    vz = (z2 - z1) / 4.0
    vs = np.sqrt(vx**2 + vy**2 + vz**2)
    print ('vtot', vs)

    orbdir = 'A' if vz > 0 else 'D'

    rs = np.sqrt(xs**2 + ys**2 + zs**2)
    height, re = self.get_height(xs, ys, zs)
    re2 = re
    print ('rs', rs, 'height', height, 're2', re2)
    
    a = np.array([xs / rs, ys / rs, zs / rs])
    b = np.array([vx / vs, vy / vs, vz / vs])
    c = np.cross(a, b)
    #print ('c', c)

    ro = self.get('near_range')
    ct = (rs**2 + ro**2 - re**2) / (2.0 * rs * ro)
    st = np.sin(np.arccos(ct))

    xe = xs + ro * (-st * c[0] - ct * a[0])
    ye = ys + ro * (-st * c[1] - ct * a[1])
    ze = zs + ro * (-st * c[2] - ct * a[2])
    rlat = np.arcsin(ze / re)
    st = np.sin(rlat)
    ct = np.cos(rlat)
    arg = (ct**2) / (ra**2) + (st**2) / (rc**2)
    re = 1.0 / np.sqrt(arg)

    print ('rlat', rlat, 're', re)
    
    #time = np.linspace(-dt * (nt // 2), dt * (nt // 2), nt) + t0
    time = np.zeros(nt)
    rng = np.zeros(nt)
    for k in range(nt):
        t = dt * (k - nt / 2)
        xs, ys, zs = spline(t0 + t)
        time[k] = t
        rng[k] = np.sqrt((xe - xs)**2 + (ye - ys)**2 + (ze - zs)**2) - ro
        #print ('\t', t, rng[k])
    print ('time', time[:3])
    print ('rng', rng[:3])
    
    d = np.polyfit(time, rng, 2)
    print ('d', d)
    rdot = d[1]
    vg = np.sqrt(ro * 2.0 * d[0])

    print ('rdot', rdot, 'vg', vg)
    
    return height, re2, vg, vs, rdot

PRM.get_velocity=get_velocity
height, re, vel,_,_ = prm_rep.get_velocity(spline_rep, t_rep[0], t_rep[2])
#SC_vel              = 7153.263621000000 
#SC_height           = 700872.176433234476 

#====calc_height_velocity rs, height, re2: 7070457.214953, 700872.119966, 6369585.094987
#====calc_height_velocity c: 0.783642, -0.594089, -0.181556
#====calc_height_velocity rlat: 0.697945
#====calc_height_velocity re: 6369279.170492
#====calc_height_velocity nt: 100
# 		t1, rng: 225430.148049, 12943.196271
# 		t1, rng: 225430.559160, 12437.676028
# 		t1, rng: 225430.970272, 11942.010446
# ...
# 		t1, rng: 225470.025841, 11134.385262
# 		t1, rng: 225470.436953, 11613.425381
# 		t1, rng: 225470.848064, 12102.343004
#====time: -20.555563 -20.144452 -19.733340...
#====rng: 12943.196271 12437.676028 11942.010446 ...
#====d: 8.112514 -8.314069 30.260371
#====rdot, vg: -8.314069 7153.264199
height, re, vel

#vtot 7593.081557320316

vtot 7593.081557320316
rs 7070457.214901702 height 700872.1199148409 re2 6369585.094986861
rlat 0.6979446746557342 re 6369279.170489837
time [-20.55556301 -20.14445175 -19.73334049]
rng [12943.19632494 12437.67610874 11942.01055035]
d [30.26037086 -8.31407475  8.11243721]
rdot -8.314074746997731 vg 7153.264204991109


(700872.1199148409, 6369585.094986861, 7153.264204991109)

In [83]:
t1 = (86400.0) * prm_rep.get('clock_start') + (prm_rep.get('nrows') - prm_rep.get('num_valid_az')) / (2.0 * prm_rep.get('PRF'))
t2 = t1 + prm_rep.get('num_patches') * prm_rep.get('num_valid_az') / prm_rep.get('PRF')
t1, t2
prm_rep.get_velocity(spline_rep, t1, t2)
#SC_vel              = 7153.263621000000 
#SC_height           = 700872.176433234476 

xs, ys, zs 3931418.56659841 3821410.3406288857 4464541.928411318
rs 7070457.097515175 height 700872.2622497436 re2 6369584.835265432
c [ 0.7836413  -0.59408937 -0.18155562]
rlat 0.6979571029166931 re 6369278.90854201
time [-20.55556301 -20.14445175 -19.73334049]
rng [12943.19337763 12437.67325908 11942.00779676]
d [30.26036597 -8.31403128  8.11243604]


(700872.2622497436,
 6369584.835265432,
 7153.263627278231,
 7593.081349211761,
 -8.314031284402429)

In [42]:
def get_velocity(prm, spline, t1, t2):
    ra = prm.get('equatorial_radius')
    rc = prm.get('polar_radius')
    dt = 200.0 / prm.get('PRF')
    nt = 100

    t0 = (t1 + t2) / 2.0
    t1 = t0 - 2.0
    t2 = t0 + 2.0

    xs, ys, zs = spline(t0)
    x1, y1, z1 = spline(t1)
    x2, y2, z2 = spline(t2)

    rs = np.sqrt(xs**2 + ys**2 + zs**2)
    vx = (x2 - x1) / 4.0
    vy = (y2 - y1) / 4.0
    vz = (z2 - z1) / 4.0
    vs = np.sqrt(vx**2 + vy**2 + vz**2)
    vtot = vs

    rlat = np.arcsin(zs / rs)
    arg = np.cos(rlat)**2 / ra**2 + np.sin(rlat)**2 / rc**2
    re = 1.0 / np.sqrt(arg)
    height = rs - re

    a = np.array([xs / rs, ys / rs, zs / rs])
    b = np.array([vx / vs, vy / vs, vz / vs])
    c = np.cross(a, b)

    ro = prm.get('near_range')
    ct = (rs**2 + ro**2 - re**2) / (2.0 * rs * ro)
    st = np.sin(np.arccos(ct))

    xe = xs + ro * (-st * c[0] - ct * a[0])
    ye = ys + ro * (-st * c[1] - ct * a[1])
    ze = zs + ro * (-st * c[2] - ct * a[2])
    rlat = np.arcsin(ze / re)
    arg = (np.cos(rlat)**2) / (ra**2) + (np.sin(rlat)**2) / (rc**2)
    re = 1.0 / np.sqrt(arg)

    time = np.linspace(-dt * (nt // 2), dt * (nt // 2), nt) + t0
    rng = np.zeros(nt)

    for k in range(nt):
        t = time[k]
        xs, ys, zs = spline(t)
        rng[k] = np.sqrt((xe - xs)**2 + (ye - ys)**2 + (ze - zs)**2) - ro

    d = np.polyfit(time, rng, 2)
    rdot = d[1]
    vg = np.sqrt(ro * 2.0 * d[2])
    return height, re, vg, vtot, rdot

# Example usage
params = {
    'equatorial_radius': 6378137.0,
    'polar_radius': 6356752.3,
    'PRF': 1679.902,  # example value
    'near_range': 850000.0  # example value
}

PRM.get_velocity=get_velocity

height, re, vg, vtot, rdot = prm_rep.get_velocity(spline_rep, t_rep[0], t_rep[2])

print(f"SC_vel = {vtot}")
print(f"SC_height = {height + re}")

SC_vel = 7593.081557320316
SC_height = 7070151.290404678


In [76]:
#====time: -20.555563 -20.144452 -19.733340...
np.diff([-20.555563, -20.144452, -19.733340])

array([0.411111, 0.411112])